# Project 2:  Lucid Titanic Sleuthing

## Part 1: Developing an Understanding of the Data

#### Based on the description of the data you read in the readme describe in your own words this data.

There are two files, named by the boat which travellers utilized for their trip. The data for each ship outlines class, survival and basic characteristics of each traveller. The Titanic data appears more straightforward and 'clean' whereas the Lusitania data will require more data manipulation to draw the same types of conclusions that Titanic clearly outlines.

#### Based on our conceptual understanding of the columns in this data set.  What are the reasonable range of the values for the Sex, Age, SibSp, Parch columns.

##### Titanic:

- Sex: M,F 
- Age: 0-100
- SibSp: 0-6
- Parch: 0-2

#### Open the data in sublime text. Is there anything that jumps out to you?

##### Titanic:

- As you move down to the bottom of the file, you see that the data is more sparse
- There are some missing values for age

##### Lusitania:

- The data appears to be rather messy and have a lot of missing values

## Part 2: Reading the Data

#### Now read the data into a Pandas DataFrame

In [1]:
import pandas as pd
import numpy as np
tdf = pd.read_csv('titanic.csv')
ldf = pd.read_csv('lusitania.csv')

#### Check that the age column doesn't have any unreasonable values 

The age column in the titanic age column appears to have reasonable values, with all numbers and a range of 0.17 to 80 (see below). The Lusitania data, however, has very unreasonable values, complete with 'months', question marks, and parenthesis.

In [2]:
print tdf['age'].unique()
print ldf['Age'].unique()
print tdf['age'].min()
print tdf['age'].max()

[ 29.     0.92   2.    30.    25.    48.    63.    39.    53.    71.    47.
  18.    24.    26.    80.      nan  50.    32.    36.    37.    42.    19.
  35.    28.    45.    40.    58.    22.    41.    44.    59.    60.    33.
  17.    11.    14.    49.    76.    46.    27.    64.    55.    70.    38.
  51.    31.     4.    54.    23.    43.    52.    16.    32.5   21.    15.
  65.    28.5   45.5   56.    13.    61.    34.     6.    57.    62.    67.
   1.    12.    20.     0.83   8.     0.67   7.     3.    36.5   18.5    5.
  66.     9.     0.75  70.5   22.5    0.33   0.17  40.5   10.    23.5   34.5
  20.5   30.5   55.5   38.5   14.5   24.5   60.5   74.     0.42  11.5   26.5 ]
['38' '37' '30' '25' '27' '48' nan '24' '19 ?' '57' '50' '56' '41' '19'
 '33' '29' '18' '20' '21' '26' '17' '58' '47' '54' '35' '43' '59' '53' '44'
 '51' '40' '49' '42' '32' '31' '34' '22' '45' '36' '29 ?' '52' '23' '60'
 '28' '16' '46' '15' '39' '63 ?' '55' '64' '53 ?' '09-months' '6' '9' '14'
 '10' '12' '?' '

#### Check for missing values.  How do you know that a value is missing?

See above. Each unique list for both ships show 'nan' in the 'age' column.

#### Does it makes sense to guess at the value?

It could make sense in some cases to guess at the value.

####  <font color='blue'>Below, I am cleaning the Titanic data.</font>

#####  <font color='green'>Title Cleaning</font>

In [3]:
# split name like the Lusitania provide, so i can think about using this to estimate age where missing
tdf['lname'] = tdf['name'].str.extract('([^\,]*)', expand=True)
tdf['title'] = tdf['name'].str.extract(',(.*?)\.', expand=True) #i had to add a '?' due to index 247 (Rothschild, Mrs. Martin (Elizabeth L. Barrett)
tdf['fname'] = tdf['name'].str.extract('\.(.*)', expand=True)

In [4]:
tdf['lname'] = [str.strip(i) for i in tdf['lname']]
tdf['title'] = [str.strip(i) for i in tdf['title']]
tdf['fname'] = [str.strip(i) for i in tdf['fname']]

In [5]:
# check uniques in title since that's really the only one I care about
print tdf['title'].unique()

['Miss' 'Master' 'Mr' 'Mrs' 'Col' 'Mme' 'Dr' 'Major' 'Capt' 'Lady' 'Sir'
 'Mlle' 'Dona' 'Jonkheer' 'the Countess' 'Don' 'Rev' 'Ms']


#####  <font color='green'>Dummy Variables - Class, Gender, Parents/Children</font>

In [6]:
#turning classes into dummy variables
tdf['class_1'] = tdf['pclass'].apply(lambda x: 1 if x == 1 else 0)
tdf['class_2'] = tdf['pclass'].apply(lambda x: 1 if x == 2 else 0)
tdf['class_3'] = tdf['pclass'].apply(lambda x: 1 if x == 3 else 0)

In [7]:
# turning genders into dummy variables (1 for male and 0 for female)
tdf['sex'] = tdf['sex'].apply(lambda x: 1 if x == 'male' else 0)

In [8]:
tdf_clean = pd.DataFrame(tdf[['title','sex','survived','age','class_1','class_2','class_3']])
tdf_clean.columns = ['t_title','t_sex','t_fate','t_age','t_class_1','t_class_2','t_class_3']

####  <font color='blue'>Below, I am cleaning the Lusitania data.</font>

In [9]:
ldf = pd.DataFrame(ldf[['Title','Sex','Fate','Age','Passenger/Crew','Status','Department/Class', 'Adult/Minor']])
ldf.columns = ['l_title','l_sex','l_fate','l_age','l_pass','l_marit_status','l_class','l_age_status']

##### <font color='green'>Age Cleaning</font>

In [10]:
# assigning all ages provided in months or as infant to the string '1', otherwise returning the l_age cell contents
ldf['age_str'] = ['1' if 'nt' in str(age) else age for age in ldf['l_age']]

In [11]:
# extracting the first 1 or 2 digit strings from the age_str column, and then casting to float
ldf['age_flt'] = ldf['age_str'].str.extract('(\d{1,2})', expand=True)
ldf['age_flt'] = [float(i) for i in ldf['age_flt']]

##### <font color='green'>Dummy Variables - Class, Gender, Fate, Marital Status, Age Status</font>

In [12]:
# remove all crew from Lusitania data since we don't have crew data for the Titanic. I'm also dropping stowaways 
# since there are only 3.
ldf = ldf.drop(ldf[ldf['l_pass'] != 'Passenger'].index)
# remove all people in data set who weren't actually on board
ldf = ldf.drop(ldf[ldf['l_fate'] == 'Not on board'].index)

In [13]:
# reassigning each class and creating 3 dummy value columns
ldf['l_class_1'] = ldf['l_class'].apply(lambda x: 1 if x == 'Saloon' else 0)
ldf['l_class_2'] = ldf['l_class'].apply(lambda x: 1 if x == 'Second' else 0)
ldf['l_class_3'] = ldf['l_class'].apply(lambda x: 1 if 'Third' in x else 0)

In [14]:
# turning genders into dummy variables (1 for male and 0 for female)
ldf['l_sex'] = ldf['l_sex'].apply(lambda x: 1 if x == 'Male' else 0)

In [15]:
# turning fate into dummy variables (1 for survived and 0 for died)
ldf['l_fate'] = ldf['l_fate'].apply(lambda x: 1 if 'Saved' in x else 0)

In [16]:
# reassigning each age status to dummy columns
ldf['adult'] = ldf['l_age_status'].apply(lambda x: 1 if x == 'Adult' else 0)
ldf['minor'] = ldf['l_age_status'].apply(lambda x: 1 if x == 'Minor' else 0)
ldf['infant'] = ldf['l_age_status'].apply(lambda x: 1 if x == 'Infant' else 0)

In [17]:
# cleaning the status column
ldf['l_marit_status_mod'] = ldf['l_marit_status'].str.replace(' \(\?\)','')
ldf['l_marit_status_mod'] = ldf['l_marit_status'].str.replace('\?','Unknown')
# replace all singles with 0 and anyone who has ever been married or will be married soon will be replaced with a 1.
# ldf['l_marit_status'] = ldf['l_marit_status'].apply(lambda x: 0 if x == 'Single')
ldf['l_marit_status_mod'] = ldf['l_marit_status_mod'].apply(lambda x: 0 if x == 'Unknown' or x == 'Single' else 1)

In [18]:
ldf_clean = pd.DataFrame(ldf[['l_title','l_sex','l_fate','age_flt','l_class_1','l_class_2','l_class_3','adult','minor','infant','l_marit_status_mod']])
ldf_clean.columns = ['l_title','l_sex','l_fate','l_age','l_class_1','l_class_2','l_class_3','adult','minor','infant','l_marit_status',]

## Part 3: Data Imputation

#### Well let’s say that it does... You likely noticed that Age has some missing values. How many are missing?

Titanic has 263 missing age entries (see below), and Lusitania has 161 missing age entries (see below).

#### For the Age of the passangers ... how would you guess at the missing values using the other data present in the CSV.

Titanic: I really wanted to group the mean of each title and then assign that mean for that group to the missing values if the title matched, but I wasted countless hours trying to do it and gave up. If I knew how to do it, that is what I would do.

Similarly for the Lusitania, I would have wanted to look at the minors, infants, adults, and take into account their marital status, and estimate an age.

However, I just took the average of the entire dataset since I couldn't figure out how to separate them into groups and keep all groups in the dataset.

#### <font color='blue'>Titanic Age Estimations</font>

In [19]:
t_null_ages = tdf_clean[(tdf_clean['t_age'] >= 0) == False]
len(t_null_ages)

263

In [20]:
# estimate ages by title, as it is the only attribute that may have a relationship with age. since
# i spent hours on trying to figure out how to do this, i gave up and just assigned the mean of the column to
# all nan values
t_dr_age = tdf_clean[(tdf_clean['t_title'] == 'Dr')]
t_dr_age_mean = np.mean(t_dr_age['t_age'])
t_master_age = tdf_clean[(tdf_clean['t_title'] == 'Master')]
t_master_age_mean = np.mean(t_master_age['t_age'])
t_miss_age = tdf_clean[(tdf_clean['t_title'] == 'Miss')]
t_miss_age_mean = np.mean(t_miss_age['t_age'])
t_mr_age = tdf_clean[(tdf_clean['t_title'] == 'Mr')]
t_mr_age_mean = np.mean(t_mr_age['t_age'])
t_mrs_age = tdf_clean[(tdf_clean['t_title'] == 'Mrs')]
t_mrs_age_mean = np.mean(t_mrs_age['t_age'])
t_ms_age = tdf_clean[(tdf_clean['t_title'] == 'Ms')]
t_ms_age_mean = np.mean(t_ms_age['t_age'])

In [21]:
t_null_ages.pivot_table('t_age',index = 't_title', aggfunc = [np.mean, np.count_nonzero, np.std, np.max, np.min], 
                      margins = True)

,mean,count_nonzero,std,amax,amin
t_title,,,,,
Dr,NaN,1.0,NaN,NaN,NaN
Master,NaN,8.0,NaN,NaN,NaN
Miss,NaN,50.0,NaN,NaN,NaN
Mr,NaN,176.0,NaN,NaN,NaN
Mrs,NaN,27.0,NaN,NaN,NaN
Ms,NaN,1.0,NaN,NaN,NaN
All,NaN,0.0,NaN,NaN,NaN


In [22]:
tdf_clean.pivot_table('t_age',index = 't_title', aggfunc = [np.mean, np.count_nonzero, np.std, np.max, np.min], 
                      margins = True)

,mean,count_nonzero,std,amax,amin
t_title,,,,,
Capt,70.000000,1.0,NaN,70.0,70.00
Col,54.000000,4.0,5.477226,60.0,47.00
Don,40.000000,1.0,NaN,40.0,40.00
Dona,39.000000,1.0,NaN,39.0,39.00
Dr,43.571429,8.0,11.731115,54.0,23.00
Jonkheer,38.000000,1.0,NaN,38.0,38.00
Lady,48.000000,1.0,NaN,48.0,48.00
Major,48.500000,2.0,4.949747,52.0,45.00
Master,5.482642,61.0,4.161554,14.5,0.33


In [23]:
tdf_clean['t_age'] = tdf_clean['t_age'].fillna(np.mean(tdf_clean['t_age']))

#### <font color='blue'>Lusitania Age Estimations</font>

In [24]:
l_null_ages = ldf_clean[(ldf_clean['l_age'] >= 0) == False]
len(l_null_ages)

161

In [25]:
ldf_clean['l_age'] = ldf_clean['l_age'].fillna(np.mean(ldf_clean['l_age']))

## Part 4: Group Statistics

#### Are there any groups that were especially adversely affected in the Titanic wreck? (justify your response numerically)

In [26]:
from scipy.stats import chi2_contingency

In [27]:
t_class_survive_pivot = tdf_clean.pivot_table(['t_class_1','t_class_2','t_class_3'],index = 't_fate', aggfunc = 'sum')
t_class_survive_pivot

,t_class_1,t_class_2,t_class_3
t_fate,,,
0,123,158,528
1,200,119,181


In [28]:
chi2_contingency(t_class_survive_pivot)

(127.85915643930328,
 1.7208259588256052e-28,
 2,
 array([[ 199.62337662,  171.19404125,  438.18258212],
        [ 123.37662338,  105.80595875,  270.81741788]]))

In [29]:
tdf_clean.pivot_table(['t_class_1','t_class_2','t_class_3'],index = 't_fate', aggfunc = 'sum', margins = True)

,t_class_1,t_class_2,t_class_3
t_fate,,,
0,123.0,158.0,528.0
1,200.0,119.0,181.0
All,323.0,277.0,709.0


Yes, the third class was adversely affected. There is a clear relationship between class and survival rate. The survival rate for the titanic was approximately 38%, where 26% of the third class survived, 43% of second class survived, and 62% of first survived. Looking at the expected survival values, first and second class actual values surpass the expectation.

#### Are there any groups that outperformed the survival of the latter group? (justify your response numerically)


In [30]:
t_class_survive_pivot_1_3 = tdf_clean.pivot_table(['t_class_1','t_class_3'],index = 't_fate', aggfunc = 'sum')
chi2_contingency(t_class_survive_pivot_1_3)

(124.62521354251798,
 6.147496811642943e-29,
 1,
 array([[ 203.75290698,  447.24709302],
        [ 119.24709302,  261.75290698]]))

In [31]:
t_class_survive_pivot_2_3 = tdf_clean.pivot_table(['t_class_2','t_class_3'],index = 't_fate', aggfunc = 'sum')
chi2_contingency(t_class_survive_pivot_2_3)

(27.772913401505178,
 1.3642281379956039e-07,
 1,
 array([[ 192.72008114,  493.27991886],
        [  84.27991886,  215.72008114]]))

In [32]:
t_class_survive_pivot_1_2 = tdf_clean.pivot_table(['t_class_1','t_class_2'],index = 't_fate', aggfunc = 'sum')
chi2_contingency(t_class_survive_pivot_1_2)

(20.772001470299045,
 5.1733792710189712e-06,
 1,
 array([[ 151.27166667,  129.72833333],
        [ 171.72833333,  147.27166667]]))

Yes, as shown above, first class outperformed both groups, whereas second class only outperformed third class.

## Part 5:  Comparative Statistics:  Lusitania

#### Are there any groups that were especially adversely affected in the Lusitania wreck? (justify your response numerically)

In [33]:
l_class_survive_pivot = ldf_clean.pivot_table(['l_class_1','l_class_2','l_class_3'],index = 'l_fate', aggfunc = 'sum')
l_class_survive_pivot

,l_class_1,l_class_2,l_class_3
l_fate,,,
0,177,372,239
1,113,229,134


In [34]:
chi2_contingency(l_class_survive_pivot)
# doesn't look like a relationship with class

(0.73911123039443172,
 0.69104135068912131,
 2,
 array([[ 180.79113924,  374.67405063,  232.53481013],
        [ 109.20886076,  226.32594937,  140.46518987]]))

In [35]:
l_sex_survive_pivot = ldf_clean.pivot_table(['l_age'],index = 'l_fate', columns = 'l_sex',aggfunc = 'count')
l_sex_survive_pivot

l_age     
l_sex      0    1
l_fate           
0        309  479
1        183  293

In [36]:
chi2_contingency(l_sex_survive_pivot)
# doesn't look like a relationship with sex

(0.044835716500665791,
 0.83230640009660906,
 1,
 array([[ 306.72151899,  481.27848101],
        [ 185.27848101,  290.72151899]]))

In [37]:
l_age_survive_pivot = ldf_clean.pivot_table(['adult','minor','infant'],index = 'l_fate', aggfunc = 'sum')
l_age_survive_pivot

,adult,infant,minor
l_fate,,,
0,686,37,65
1,423,10,43


In [38]:
chi2_contingency(l_age_survive_pivot)
# better relationship with age, but doesnt quite have a small enough p-value.

(5.6971821063639227,
 0.057925877887931951,
 2,
 array([[ 691.37025316,   29.30063291,   67.32911392],
        [ 417.62974684,   17.69936709,   40.67088608]]))

#### Are there any groups that outperformed the survival of the latter group? (justify your response numerically)



In [39]:
l_age_survive_pivot_a_m = ldf_clean.pivot_table(['adult','minor'],index = 'l_fate', aggfunc = 'sum')
l_age_survive_pivot_a_m

,adult,minor
l_fate,,
0,686,65
1,423,43


In [40]:
chi2_contingency(l_age_survive_pivot_a_m)

(0.056460726244854098,
 0.81217997975577561,
 1,
 array([[ 684.35414955,   66.64585045],
        [ 424.64585045,   41.35414955]]))

In [41]:
l_age_survive_pivot_a_i = ldf_clean.pivot_table(['adult','infant'],index = 'l_fate', aggfunc = 'sum')
l_age_survive_pivot_a_i

,adult,infant
l_fate,,
0,686,37
1,423,10


In [42]:
chi2_contingency(l_age_survive_pivot_a_i)

(4.7786572523516728,
 0.028814580347396229,
 1,
 array([[ 693.60467128,   29.39532872],
        [ 415.39532872,   17.60467128]]))

In [43]:
l_age_survive_pivot_m_i = ldf_clean.pivot_table(['minor','infant'],index = 'l_fate', aggfunc = 'sum')
l_age_survive_pivot_m_i

,infant,minor
l_fate,,
0,37,65
1,10,43


In [44]:
chi2_contingency(l_age_survive_pivot_m_i)

(4.2117125919832823,
 0.040145785953812727,
 1,
 array([[ 30.92903226,  71.07096774],
        [ 16.07096774,  36.92903226]]))

Infants fared the worst out of the three groups, with survival rate of 21%.

#### What does the group-wise survival rate imply about circumstances during these two accidents?

For the Titanic, the highest chance of survival depended on your purchased ticket, whereas Lusitania did not have a clear relationship, besides the fact that individuals with the ability to survive had a chance at survival.